# Main Telescope EFD Analysis

This notebook is used to extract the positions, velocities, acceleration, jerk and motor torques for each axis of the Main Telescope for a given time interval. By inputting a time interval, this notebook allows the rapid determination of what caused the fault, particularily in the case of motor slippage, or if a drive was commanded in an unstable fashion. <br>

It is expected that the user interacts with the Bokeh plots to better pinpoint the issue they're searching for. The plot ranges as created will most likely be too large, or contain too much data to be useful with zero manipulation of the axes/zooms etc. <br>

This notebook extracts data from the DM-EFD using [aioinflux](https://aioinflux.readthedocs.io/en/stable/index.html), a Python client for InfluxDB, and proceed with data analysis using Pandas dataframes. 

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

In addition to `aioinflux`, you'll need to install `pandas`, `numpy` and `matplotlib` to run this notebook.

In [1]:
import matplotlib
%matplotlib widget
from matplotlib import pylab as plt
import aioinflux
import getpass
import pandas as pd
import asyncio
import numpy as np
from astropy.time import Time, TimeDelta

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
output_notebook()
from bokeh.models import Span, Label

from lsst_efd_client import EfdClient, resample

Loading BokehJS ...

In [ ]:
# Change client if using USDF. 
client = EfdClient('summit_efd')
client.output = 'dataframe'

We'll access the DM-EFD instance deployed at the summit. You need to be on site or connected to the NOAO VPN. 

## Declare timestamps used for EFD queries

In [ ]:
### Example 
t1 = Time('2023-03-08T03:40:01', scale='utc') 
window = TimeDelta(300, format='sec')
t2=t1+window

In [ ]:
# Declare offset to tai
# Used in converting pointing vector data which is in TAI
utc_to_tai_offset = TimeDelta(37, format='sec')

In [ ]:
# Find images taken during the time interval
image_info = await client.select_time_series("lsst.sal.GenericCamera.logevent_endReadout", 
                                             ["imageNumber", "timestampAcquisitionStart", "timestampEndOfReadout", "requestedExposureTime"], 
                                             t1, t2)
#print(image_info)

In [ ]:
image_info

In [ ]:
# time check verification
print(t1.isot)
print(t1.datetime64)
print(t2.isot)

In [ ]:
# Useful code snippet on how to convert the time strings to a readable format
# help(pd.to_datetime(commanded_az_ATPng['private_sndStamp'][0], unit='s'))

In [ ]:
def show_image_boundaries(image_info, yaxis_data):
    if len(image_info) > 0:
        # Generic camera selection: #0 for Wide Camera, #1 for Narrow Camera & #2 for Fast Camera (DIMM) 
        cam = 0
        for l in range(cam, len(image_info), 3):
            # NEED to check the time tai to utc in plot
            start = Time(image_info.timestampAcquisitionStart[l], format='unix', scale='tai') + utc_to_tai_offset
            #finish = Time(image_info.timestampEndOfReadout[l], format='unix', scale='tai') # issue with timestamps in astropy
            finish = start+TimeDelta(image_info.requestedExposureTime[l], format='sec')  # workaround
            # print(f"start for {l} is {start.isot}")
            # print(f"Exposure time for {l} is {image_info.requestedExposureTime[l]}")
            # print(f"finish is for {l} is {finish.isot}")
            start_vline = Span(location=start.datetime64, dimension='height', line_color='purple', line_width=0.5, line_dash='dashed')
            finish_vline = Span(location=finish.datetime64, dimension='height', line_color='blue', line_width=0.5, line_dash='dashed')
            
            ylabel_pos = np.median(yaxis_data)
            seq_num_label = Label(x=start.datetime64, y=ylabel_pos, text=str(image_info.imageNumber[l]))

            p.add_layout(start_vline)
            p.add_layout(finish_vline)
            p.add_layout(seq_num_label)

# Azimuth Analysis

In [ ]:
# query angle of azimuth (4 individual encoder heads)
az = await client.select_time_series("lsst.sal.MTMount.encoder",
                                      ["azimuthEncoderPosition0", "azimuthEncoderPosition1", "azimuthEncoderPosition2", "azimuthEncoderPosition3"], 
                                      t1, t2)

In [ ]:
# query the angle commanded by the MT Pointing Component 
commanded_az_ATPng = await client.select_time_series("lsst.sal.MTMount.command_trackTarget", 
                                                     ["azimuth","private_sndStamp", "private_rcvStamp"], 
                                                      t1, t2)

## Azimuth Position - Plot commanded position (by pointing component), Calculated encoder position (by MTMount)

In [ ]:
# left plot axis range
yr_cen=np.median(commanded_az_ATPng['azimuth'])
dy=1.1*(np.max(commanded_az_ATPng['azimuth'])- np.min(commanded_az_ATPng['azimuth']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Azimuth [deg]"
p.xaxis.axis_label = "Time"
p.title = "AZ Position vs Time \n"


# Measured AZ from encoder0
p.cross(x=(Time(az.index.values)).value, 
        y=az['azimuthEncoderPosition0'], 
        color='red', alpha=0.5, 
        legend_label='MTMount Measured Az Position')
# MTMount Commanded AZ from MTPtng
p.line(x=(Time(commanded_az_ATPng.index.values)).value, 
       y=commanded_az_ATPng['azimuth'], 
       color='green', alpha=0.7,
       legend_label='MTPtng Commanded Target Az')

# plot image boundaries
show_image_boundaries(image_info, commanded_az_ATPng['azimuth'])

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

## Azimuth velocity 

In [ ]:
# Measured Velocity
measured_vel_az = await client.select_time_series("lsst.sal.MTMount.azimuth", 
                                                  ["actualVelocity", "demandVelocity"], 
                                                  t1, t2)

# Commanded velocity from MTPointing Component
commanded_vel_az_ATPng = await client.select_time_series("lsst.sal.MTMount.command_trackTarget", 
                                                         ["azimuthVelocity","private_sndStamp", "private_rcvStamp"], 
                                                         t1, t2)

In [ ]:
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.title = "AZ Velocity and Position vs Time \n"
p.yaxis.axis_label = "Azimuth [deg]"
p.xaxis.axis_label = "Time"

# AZ Positions
# Measured AZ from encoder0
p.cross(x=(Time(az.index.values)).value, 
        y=az['azimuthEncoderPosition0'], 
        color='red', alpha=0.5, 
        legend_label='MTMount Measured Az Position')
# MTMount Commanded AZ from MTPtng
p.line(x=(Time(commanded_az_ATPng.index.values)).value, 
       y=commanded_az_ATPng['azimuth'], 
       color='green', alpha=0.7,
       legend_label='MTPtng Commanded Target Az')

# AZ Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity [deg/s]'), 'right')
# Measured AZ velocity
p.cross(x=(Time(measured_vel_az.index.values)).value, 
        y=measured_vel_az['actualVelocity'], 
        color='blue', alpha=0.5, y_range_name='Velocity', 
        legend_label='MTMount Measured Az Velocity')

# vs commanded AZ velocity
p.line(x=(Time(commanded_vel_az_ATPng.index.values)).value, 
       y=commanded_vel_az_ATPng['azimuthVelocity'], 
       color='orange', alpha=0.7, y_range_name='Velocity', 
       legend_label='MTPtng Commanded Target Az Velocity')


p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

## Azimuth Torque 

In [ ]:
# Demanded and Measured Torques

az_torque_measured = await client.select_time_series("lsst.sal.MTMount.azimuth", 
                                                     ["actualTorque", ], 
                                                     t1, t2)

In [ ]:
# Torque Plot

yr_cen=np.median(measured_vel_az['actualVelocity'])
dy=1.1*(np.max(measured_vel_az['actualVelocity'])- np.min(measured_vel_az['actualVelocity']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Velocity [deg/s]"
p.xaxis.axis_label = "Time"
p.title = "AZ Torque and Velocity vs Time \n"

# AZ velocity
p.cross(x=(Time(measured_vel_az.index.values)).value, 
        y=measured_vel_az['actualVelocity'], 
        color='blue', alpha=0.5, 
        legend_label='MTMount Measured Az Velocity')
# p.cross(x=(Time(measured_vel_az.index.values)).value, 
#         y=measured_vel_az['demandVelocity'], 
#         color='pink', alpha=0.5, y_range_name='Velocity', 
#         legend_label='MTMount Demand Az Velocity')

# vs commanded AZ velocity
p.line(x=(Time(commanded_vel_az_ATPng.index.values)).value, 
       y=commanded_vel_az_ATPng['azimuthVelocity'], 
       color='orange', alpha=0.7, 
       legend_label='MTPtng Commanded Target Az Velocity')


# Measured AZ Torques
yr_cen=np.median(az_torque_measured['actualTorque'])
dy=(np.max(az_torque_measured['actualTorque']) - np.min(az_torque_measured['actualTorque']))/2
p.extra_y_ranges = {'Torque': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque [A]'), 'right')

p.line(x=(Time(az_torque_measured.index.values)).value, 
       y=az_torque_measured['actualTorque'], 
       color='green', alpha=0.5, y_range_name='Torque', 
       legend_label='MTMount AZ Measured Torque')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Acceleration and jerk 

In [ ]:
# Measured Acceleration
measured_acc_az = await client.select_time_series("lsst.sal.MTMount.azimuth", 
                                                  ["actualAcceleration"], 
                                                  t1, t2)


In [ ]:
# No jerk available in EFD  - Derivative
acc_az = np.array(measured_acc_az.values.tolist())[:,0]
times=(measured_acc_az.index - measured_acc_az.index[0]).total_seconds()
acc_derivative = np.gradient(acc_az, times)
calculated_jerk_az=pd.DataFrame({'times': measured_acc_az.index, 'jerk':acc_derivative})

In [ ]:
# Acceleration and Jerk Plot

yr_cen=np.median(measured_acc_az['actualAcceleration'])
dy=1.1*(np.max(measured_acc_az['actualAcceleration'])- np.min(measured_acc_az['actualAcceleration']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Acceleration [deg/s\u00b2]"
p.xaxis.axis_label = "Time"
p.title = "AZ Acceleration and Jerk vs Time \n"

# AZ acceleration
# Velocity
# p.cross(x=(Time(measured_vel_az.index.values)).value, 
#         y=measured_vel_az['actualVelocity'], 
#         color='red', alpha=0.5, 
#         legend_label='MTMount Measured Az Velocity')

# Acceleration
p.cross(x=(Time(measured_acc_az.index.values)).value, 
       y=measured_acc_az['ActualAcceleration'], 
       color='blue', alpha=0.7, 
       legend_label='MTMount Measured Az Velocity')


# Calculated Jerk
yr_cen=np.median(calculated_jerk_az['jerk'])
dy=(np.max(calculated_jerk_az['jerk']) - np.min(calculated_jerk_az['jerk']))/2
p.extra_y_ranges = {'Jerk': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Jerk', axis_label='Jerk [deg/s\u00b3]'), 'right')

p.line(x=(Time(calculated_jerk_az['times'])).value, 
       y=calculated_jerk_az['jerk'], 
       color='green', alpha=0.5, y_range_name='Jerk', 
       legend_label='MTMount AZ Jerk')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

# Elevation Axis Analysis

In [ ]:
# query angle of elevation (4 individual encoder heads)
el = await client.select_time_series("lsst.sal.MTMount.encoder",
                                      ["elevationEncoderPosition0", "elevationEncoderPosition1", "elevationEncoderPosition2", "elevationEncoderPosition3"], 
                                      t1, t2)

In [ ]:
# query the angle commanded by the MT Pointing Component 
commanded_el_ATPng = await client.select_time_series("lsst.sal.MTMount.command_trackTarget", 
                                                     ["elevation","private_sndStamp", "private_rcvStamp"], 
                                                      t1, t2)

## Elevation Position -  Plot commanded position (by pointing component), Calculated encoder position (by MTMount)

In [ ]:
#derive principal (left) plot axis range
yr_cen=np.median(commanded_el_ATPng['elevation'])
dy=1.1*(np.max(commanded_el_ATPng['elevation'])- np.min(commanded_el_ATPng['elevation']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Elevation [deg]"
p.xaxis.axis_label = "Time"
p.title = "El Position vs Time \n"

# Measured AZ from encoder0
p.cross(x=(Time(el.index.values)).value, 
        y=el['elevationEncoderPosition0'], 
        color='red', alpha=0.5, 
        legend_label='MTMount Measured EL Position')
# MTMount Commanded AZ from MTPtng
p.line(x=(Time(commanded_el_ATPng.index.values)).value, 
       y=commanded_el_ATPng['elevation'], 
       color='green', alpha=0.7,
       legend_label='MTPtng Commanded Target El')

# plot image boundaries
show_image_boundaries(image_info, commanded_el_ATPng['elevation'])

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Elevation velocity 

In [ ]:
# Measured Velocity
measured_vel_el = await client.select_time_series("lsst.sal.MTMount.elevation", 
                                                  ["actualVelocity", "demandVelocity"], 
                                                  t1, t2)

# Commanded velocity from MTPointing Component
commanded_vel_el_ATPng = await client.select_time_series("lsst.sal.MTMount.command_trackTarget", 
                                                         ["elevationVelocity","private_sndStamp", "private_rcvStamp"], 
                                                         t1, t2)

In [ ]:
p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Elevation [deg]"
p.xaxis.axis_label = "Time"
p.title = "EL Velocity and Position vs Time \n"


# EL Positions
# Measured EL from encoder0
p.cross(x=(Time(el.index.values)).value, 
        y=el['elevationEncoderPosition0'], 
        color='red', alpha=0.5, 
        legend_label='MTMount Measured EL Position')
# MTMount Commanded EL from MTPtng
p.line(x=(Time(commanded_el_ATPng.index.values)).value, 
       y=commanded_el_ATPng['elevation'], 
       color='green', alpha=0.7,
       legend_label='MTPtng Commanded Target El')

# EL Velocities
p.extra_y_ranges = {'Velocity': Range1d(start=-0.1, end=0.1)}
p.add_layout(LinearAxis(y_range_name='Velocity', axis_label='Velocity [deg/s]'), 'right')
# Measured AZ velocity
p.cross(x=(Time(measured_vel_az.index.values)).value, 
        y=measured_vel_el['actualVelocity'], 
        color='blue', alpha=0.5, y_range_name='Velocity', 
        legend_label='MTMount Measured El Velocity')

# vs commanded AZ velocity
p.line(x=(Time(commanded_vel_el_ATPng.index.values)).value, 
       y=commanded_vel_el_ATPng['elevationVelocity'], 
       color='orange', alpha=0.7, y_range_name='Velocity', 
       legend_label='MTPtng Commanded Target El Velocity')


p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
show(p)

## Elevation Torque

In [ ]:
# Demanded and Measured Torques

el_torque_measured = await client.select_time_series("lsst.sal.MTMount.elevation", 
                                                     ["actualTorque", ], 
                                                     t1, t2)

In [ ]:
# Torque Plot

yr_cen=np.median(measured_vel_el['actualVelocity'])
dy=1.1*(np.max(measured_vel_el['actualVelocity'])- np.min(measured_vel_el['actualVelocity']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Velocity [deg/s]"
p.xaxis.axis_label = "Time"
p.title = "EL Torque and Velocity vs Time \n"


# EL velocity
p.cross(x=(Time(measured_vel_el.index.values)).value, 
        y=measured_vel_el['actualVelocity'], 
        color='blue', alpha=0.5, 
        legend_label='MTMount Measured El Velocity')
# p.cross(x=(Time(measured_vel_az.index.values)).value, 
#         y=measured_vel_az['demandVelocity'], 
#         color='pink', alpha=0.5, y_range_name='Velocity', 
#         legend_label='MTMount Demand Az Velocity')

# vs commanded EL velocity
p.line(x=(Time(commanded_vel_el_ATPng.index.values)).value, 
       y=commanded_vel_el_ATPng['elevationVelocity'], 
       color='orange', alpha=0.7, 
       legend_label='MTPtng Commanded Target el Velocity')


# Measured EL Torques
yr_cen=np.median(el_torque_measured['actualTorque'])
dy=(np.max(el_torque_measured['actualTorque']) - np.min(el_torque_measured['actualTorque']))/2
p.extra_y_ranges = {'Torque': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Torque', axis_label='Torque [A]'), 'right')

#p.line(x=(Time(az_motor1_torque_demand.index.values)).value, y=az_motor1_torque_demand['azimuthMotor1Torque'], color='orange', alpha=0.7, y_range_name='Torque', legend_label='ATMCS Commanded Motor 1 Torque')
p.line(x=(Time(el_torque_measured.index.values)).value, 
       y=el_torque_measured['actualTorque'], 
       color='green', alpha=0.5, y_range_name='Torque', 
       legend_label='MTMount EL Measured Torque')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)

## Acceleration and jerk 

In [ ]:
# Measured Acceleration
measured_acc_el = await client.select_time_series("lsst.sal.MTMount.elevation", 
                                                  ["actualAcceleration"], 
                                                  t1, t2)


In [ ]:
# No jerk available in EFD  - Derivative
acc_el = np.array(measured_acc_el.values.tolist())[:,0]
times=(measured_acc_el.index - measured_acc_el.index[0]).total_seconds()
acc_derivative = np.gradient(acc_el, times)
calculated_jerk_el=pd.DataFrame({'times': measured_acc_el.index, 'jerk':acc_derivative})

In [ ]:
# Acceleration and Jerk Plot

# yr_cen=np.median(measured_acc_az['actualAcceleration'])
# dy=1.1*(np.max(measured_acc_az['actualAcceleration'])- np.min(measured_acc_az['actualAcceleration']))

p = figure(x_axis_type='datetime', y_range=(yr_cen-dy, yr_cen+dy), plot_width=800, plot_height=400)
p.yaxis.axis_label = "Acceleration [deg/s\u00b2]"
p.xaxis.axis_label = "Time"
p.title = "EL Acceleration and Jerk vs Time \n"

# AZ acceleration
# Velocity
# p.cross(x=(Time(measured_vel_el.index.values)).value, 
#         y=measured_vel_el['actualVelocity'], 
#         color='red', alpha=0.5, 
#         legend_label='MTMount Measured EL Velocity')

# Acceleration
# vs actual acceleration
p.cross(x=(Time(measured_acc_el.index.values)).value, 
       y=measured_acc_el['ActualAcceleration'], 
       color='blue', alpha=0.7, 
       legend_label='MTMount Measured EL Acceleration')


# Calculated Jerk
yr_cen=np.median(calculated_jerk_el['jerk'])
dy=(np.max(calculated_jerk_el['jerk']) - np.min(calculated_jerk_el['jerk']))/2
p.extra_y_ranges = {'Jerk': Range1d(start=yr_cen-dy, end=yr_cen+dy)}
p.add_layout(LinearAxis(y_range_name='Jerk', axis_label='Jerk [deg/s\u00b3]'), 'right')

p.line(x=(Time(calculated_jerk_el['times'])).value, 
       y=calculated_jerk_el['jerk'], 
       color='green', alpha=0.5, y_range_name='Jerk', 
       legend_label='MTMount EL Jerk')

p.legend.location = 'bottom_left'
p.legend.click_policy = 'hide'
show(p)